### This notebook includes codes for baseline scenario in Mandal et al. (2024) paper. 
### The baseline scenario is vrptw with 2 indices. Please note that modules and functions are listed in util_baseline.py script

#### First we import the required packages and modules from util_baseline.py script

In [1]:
import pandas as pd
from util_baseline import generate_E_star, solve_model

#### Data is imported from excel file. We also add another row for the ending node. The ending with its attributes are added to the last row of df

In [2]:
# df = pd.read_excel("C106_50.xlsx")
# df = pd.read_excel("C101_50.xlsx")
df = pd.read_excel("R109_25.xlsx")
# Adding ending node depot
new_row = df.iloc[0].copy() 
new_row["NO."] = len(df) + 1
new_row_df = pd.DataFrame([new_row])
df = pd.concat([df, new_row_df], ignore_index=True)

#### Preprocessing data

In [3]:
# Vehicle maximum time and capacity capacity


# extracting data of customers and depots from the dataframe
node_data = {}
for idx, row in df.iterrows():
    cid = int(row['NO.'])  # or keep as float if you prefer
    node_data[cid] = {
        'x':        float(row['XCOORD.']),
        'y':        float(row['YCOORD.']),
        'demand':   float(row['DEMAND']),
        'early':    float(row['READYTime']),
        'late':      float(row['DUEDATE']),
        'service':  float(row['SERVICETIME'])
    }

d0 = 200.0
t0 = node_data[1]["late"] # this is based on Solomon dataset. The total time is equal to uppper bound of depot time window


# Getting time windows based on bedget remaining logic
for u in node_data:
    node_data[u]["t_plus_budget"] = t0 - node_data[u]["early"]  # Adjusted t^+ for budget logic
    node_data[u]["t_minus_budget"] = t0 - node_data[u]["late"]  # Adjusted t^- for budget logic


# Indices for depots
alpha = 1
bar_alpha = len(node_data)  

# Extracting all customers from all nodes
all_nodes = list(node_data.keys())  # e.g. [0,1,2,3,4,5]
customers = [n for n in all_nodes if n not in (alpha, bar_alpha)]

#### Creating feasible arcs for the model. The infeasible arcs in terms of time windows and capacity are excluded from the network

In [4]:
E_star = generate_E_star(all_nodes, node_data, alpha, bar_alpha, d0)

#### Solving the objective function model

In [5]:
objective_value = solve_model(all_nodes, customers, node_data, E_star, t0, d0, alpha, bar_alpha)

Set parameter Username
Set parameter LicenseID to value 2602720
Academic license - for non-commercial use only - expires 2025-12-22
Set parameter TimeLimit to value 1000
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[x86] - Darwin 22.1.0 22A400)

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
TimeLimit  1000

Optimize a model with 896 rows, 464 columns and 3174 nonzeros
Model fingerprint: 0x5edad496
Variable types: 54 continuous, 410 integer (410 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [7e+00, 6e+01]
  Bounds range     [1e+00, 2e+02]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective 1246.1601802
Presolve removed 137 rows and 14 columns
Presolve time: 0.02s
Presolved: 759 rows, 450 columns, 5825 nonzeros
Variable types: 50 continuous, 400 integer (400 binary)

Root relaxation: objective 3.499679e+02, 111 ite